In [1]:
pip install keras-tuner


   ---------------------------------------- 0.0/128.9 kB ? eta -:--:--
   --- ------------------------------------ 10.2/128.9 kB ? eta -:--:--
   ------------------- -------------------- 61.4/128.9 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 128.9/128.9 kB 1.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# 1.Build a deep learning model to classify the mnist digits dataset with Batch Normalization.

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load and preprocess the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# One-hot encode the labels
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

# Build the deep learning model with Batch Normalization
model = models.Sequential()

# Input layer
model.add(layers.Flatten(input_shape=(28, 28)))

# Hidden layers with Batch Normalization
model.add(layers.Dense(128))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.Dense(128))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

# Output layer
model.add(layers.Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Display the model summary
model.summary()

# Train the model
history = model.fit(train_images, train_labels, epochs=10, batch_size=64, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')

# Save the model
model.save("mnist_model_with_batch_norm.h5")


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 batch_normalization_1 (Bat  (None, 128)               512       
 chNormalization)                                       

C:\Users\hp\anaconda3\envs\new_env\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# 2.Build a Feed Forward Neural Network for any problems with keras tuner.

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

# Define the model building function
def build_model(hp):
    model = keras.Sequential()

    # Tune the number of hidden layers
    for i in range(hp.Int('num_layers', 2, 5)):
        # Tune the number of units in each hidden layer
        model.add(layers.Dense(units=hp.Int(f'units_{i}', 32, 128, 32),
                               activation='relu'))

    # Output layer
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

# Load a synthetic dataset (adapt this to your specific problem)
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Flatten the images
x_train = x_train.reshape((x_train.shape[0], -1))
x_test = x_test.reshape((x_test.shape[0], -1))

# Define the Keras Tuner RandomSearch tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Number of hyperparameter combinations to try
    directory='my_dir',  # Directory to store the search results
    project_name='my_project'  # Name for the project
)
# Perform hyperparameter tuning
tuner.search(x_train, y_train, epochs=5, validation_split=0.2)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model on the test set
test_loss, test_acc = best_model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')


Trial 5 Complete [00h 00m 33s]
val_accuracy: 0.10050000250339508

Best val_accuracy So Far: 0.10050000250339508
Total elapsed time: 00h 02m 30s
313/313 [==============================] - 1s 2ms/step - loss: -33961453551616.0000 - accuracy: 0.1000
Test accuracy: 0.10000000149011612
